# Test Local Streaming Setup

In [10]:
import asyncio

import aiohttp
import nest_asyncio
from typing import Any

In [2]:
nest_asyncio.apply()
sample_payload = {
    "payload": {
        "data": ["Apple", "Samsung", "OnePlus", "Google"]
    }
}
request_ep = "http://localhost:8080/python/produce"
response_ep = "http://localhost:8080/python/consume"

In [11]:
# Async POST and GET polling using aiohttp
poll_interval = 1 # Seconds between GET polls
max_polls = 10 # Maximum number of GET polls per request_id

async def poll_for_result(session: aiohttp.ClientSession, request_id: str) -> dict[str, Any] | None:
    """Polls the response endpoint for the result corresponding to the given request_id."""
    for attempt in range(max_polls):
        async with session.get(f"{response_ep}/{request_id}") as resp:
            if resp.status == 200:
                data = await resp.json()
                if data.get("status") == "completed":
                    print(f"Result for request_id {request_id}: {data}")
                    return data
                else:
                    print(f"Response for request_id {request_id} malformed.")
            else:
                print(f"Attempt {attempt + 1}: Result not ready yet for request_id {request_id}. Retrying in {poll_interval} seconds...")
        await asyncio.sleep(poll_interval)
    print(f"Max polls reached for request_id {request_id}. Moving to next request.")
    return None


async def post_and_poll(N: int = 2) -> None:
    async with aiohttp.ClientSession() as session:
        tasks  = []
        for i in range(N):
            async with session.post(request_ep, json=sample_payload) as resp:
                if resp.status == 202:
                    data = await resp.json()
                    request_id = data.get("request_id")
                    print(f"Request {i + 1}: Sent payload. Received request_id: {request_id}")
                    if request_id:
                        tasks.append(poll_for_result(session, request_id))
                    else:
                        print(f"No request_id received. Status code: {resp.status}")
        # Running all polling tasks concurrently
        results = await asyncio.gather(*tasks)

In [12]:
# Run the async function
n = 2 # Number of times to send the request
asyncio.run(post_and_poll(N=n))

Request 1: Sent payload. Received request_id: e9133042-3f30-4590-ab7e-c14e9259d093
Request 2: Sent payload. Received request_id: ad738b51-ec34-4f55-b3ca-778c494ede3c
Attempt 1: Result not ready yet for request_id ad738b51-ec34-4f55-b3ca-778c494ede3c. Retrying in 1 seconds...
Attempt 1: Result not ready yet for request_id e9133042-3f30-4590-ab7e-c14e9259d093. Retrying in 1 seconds...
Result for request_id e9133042-3f30-4590-ab7e-c14e9259d093: {'request_id': 'e9133042-3f30-4590-ab7e-c14e9259d093', 'status': 'completed', 'result': {'processed_data': "Processed ['Apple', 'Samsung', 'OnePlus', 'Google']"}}
Attempt 2: Result not ready yet for request_id ad738b51-ec34-4f55-b3ca-778c494ede3c. Retrying in 1 seconds...
Attempt 3: Result not ready yet for request_id ad738b51-ec34-4f55-b3ca-778c494ede3c. Retrying in 1 seconds...
Result for request_id ad738b51-ec34-4f55-b3ca-778c494ede3c: {'request_id': 'ad738b51-ec34-4f55-b3ca-778c494ede3c', 'status': 'completed', 'result': {'processed_data': "Pro